# Практическая часть курсовой работы "Полнотекстовая разметка фреймовой информации (на материале русского ФреймБанка)»

*Ксения Петухова, БКЛ182*

Импортируем все необходимые для работы модули.

In [ ]:
import pandas as pd
from numpy import median
from nltk import word_tokenize, sent_tokenize
import os
import re
from tqdm import tqdm
import ufal.udpipe
global model
import codecs
import io 
import time
from datetime import datetime
import sqlite3
import sys
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import ast
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
import numpy as np
import tqdm.notebook as tq
import pandas as pd
from pymorphy2 import MorphAnalyzer
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from collections import Counter
from sklearn.cluster import KMeans
from collections import Counter

# Подготовка данных

Для удобства необходимые для дальнейшей работы данные были переведены из формата базы данных sql в таблицы формата csv. Были созданы следующие файлы:
- targets.csv (таблица со списком слов, активирующих семантические фреймы, и их типичными синтаксическими реализациями)
- framebank_examples.csv (таблица с примерами употребления конструкций)
- expl_framebank.csv (таблица со всеми участниками из примеров таблицы из предыдущего пункта с размеченными семантическими ролями)
- baseline_df_4.csv (таблица со всеми размеченными во фреймбанке участниками с разметкой морфосинтаксических характеристик и индексом позиции участника в стандартной реализации конструкции)

Приводим примеры употребления конструкций к формату разметки Universal Dependencies (https://universaldependencies.org) создаем таблицу df_framebank_ud. Модель можно скачать по ссылке: https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3131

In [ ]:
model = ufal.udpipe.Model.load('russian-syntagrus-ud-2.5-191206.udpipe')
tokenizer = model.newTokenizer(model.DEFAULT)
conlluOutput = ufal.udpipe.OutputFormat.newOutputFormat("conllu")
sentence = ufal.udpipe.Sentence()
error = ufal.udpipe.ProcessingError()

In [ ]:
def tag(text, model):
    tokenizer.setText(text)
    tokenizer.nextSentence(sentence, error)
    model.tag(sentence, model.DEFAULT)
    model.parse(sentence, model.DEFAULT)
    return conlluOutput.writeSentence(sentence)

Функции, необходимые для унификации символов юникода для сокращения рабочего числа букв в дальнейшей обработке текста (код для унификации взят отсюда: https://github.com/TatianaShavrina/taiga/blob/master/tagging_pipeline/unify.py)

In [ ]:
def open_text(text):
    text = unify_sym(text)
    return(text)

def open_lines(text):
    lines = []
    for line in text:
        lines.append(unify_sym(line))
    return(lines)

def list_replace(search,replace, text):
   
    for c in search:
        if c in text:
            text = text.replace(c, replace)
    return text 

def unify_sym(text): 
    text = list_replace('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019','\u0022',text)
    text = list_replace('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF','\u2003\u002D\u002D\u2003',text)
    text = list_replace('\u2010\u2011','\u002D',text)
    text = list_replace('\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000','\u2002',text)
    text = re.sub('\u2003\u2003','\u2003',text)
    text = re.sub('\t\t','\t',text)
    text = list_replace('\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062','.',text)
    text = list_replace('\u2217','\u002A',text)
    text = list_replace('…','...',text)
    text = list_replace('\u2241\u224B\u2E2F\u0483','\u223D',text)
    text = list_replace('\u00C4','A',text)
    text = list_replace('\u00E4','a',text)
    text = list_replace('\u00CB','E',text)
    text = list_replace('\u00EB','e',text)
    text = list_replace('\u1E26','H',text)
    text = list_replace('\u1E27','h',text)
    text = list_replace('\u00CF','I',text)
    text = list_replace('\u00EF','i',text)
    text = list_replace('\u00D6','O',text)
    text = list_replace('\u00F6','o',text)
    text = list_replace('\u00DC','U',text)
    text = list_replace('\u00FC','u',text)
    text = list_replace('\u0178','Y',text)
    text = list_replace('\u00FF','y',text)
    text = list_replace('\u00DF','s',text)
    text = list_replace('\u1E9E','S',text)
    currencies = list('\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192')
    alphabet = list('\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-*&^%$#@!~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
    alphabet.append("'")
    text1 = ""
    for sym in text:
        if sym in alphabet:
            text1+=sym
        else:
            if sym in currencies:
                text1+=sym
            else:
                pass
        
    return(text1)

In [ ]:
def udpipe_examples_txt(path):
    framebank_examples = pd.read_csv(path)
    example_texts = list(framebank_examples.example)
    example_ids = list(framebank_examples.example_id)
    example_id_for_sentence = []
    newpath = '/Users/kseniapetuhova/Desktop/'
    name = 'udpipe_examples_framebank_2'
    for ex in example_texts:
        index_ex = example_texts.index(ex)
        out = open(os.path.join(newpath, name), 'a', encoding='utf8')
        text = unify_sym(ex)
        slist = sent_tokenize(text)
        for s in slist:
            s = tag(s, model)
            out.write(s)
            example_id_for_sentence.append(example_ids[index_ex])

        out.close()

In [ ]:
udpipe_examples_txt('framebank_examples.csv')

In [ ]:
def udpipe_examples_df():
    list_sent_id = []
    list_text_id = []
    list_text = []
    ud_texts = open('/Users/kseniapetuhova/Desktop/udpipe_examples_framebank_2','r', encoding='utf8').read()
    ud_sents = ud_texts.split('\n\n')
    del ud_sents[-1]
    for i in range(len(ud_sents)):
        list_sent_id.append(i)
        list_text_id.append(example_id_for_sentence[i])
        list_text.append(ud_sents[i])

    df_framebank_ud = pd.DataFrame()
    df_framebank_ud['sent_id'] = list_sent_id
    df_framebank_ud['text_id'] = list_text_id
    df_framebank_ud['text'] = list_text
    return df_framebank_ud

In [ ]:
df_framebank_ud = udpipe_examples_df()
df_framebank_ud

,sent_id,text_id,text
0,0,4,# newdoc\n# newpar\n# sent_id = 1\n# text = Во...
1,1,4,# sent_id = 2\n# text = Они называли Шахова зл...
2,2,4,# sent_id = 3\n# text = Ну а Пермяков с Пашути...
3,3,4,# sent_id = 4\n# text = Никого они не могли на...
4,4,4,# sent_id = 5\n# text = Знай работали себе на ...
...,...,...,...
613809,613809,143528,# sent_id = 613810\n# text = Всё было когда-то...
613810,613810,143528,# sent_id = 613811\n# text = Какие там контрол...
613811,613811,143528,"# sent_id = 613812\n# text = Он про отца-то, р..."
613812,613812,143528,# sent_id = 613813\n# text = Это было ясно.\n1...


# Идентификация слов (таргетов), активирующих фреймы.

Для каждого предложения из таблицы df_framebank_ud смотрим, есть ли в нем таргет (слово, активирующее фрейм) из таблицы таргетов "targets.csv". Если нет, то ищем вершину предложения, проверяем, что она является глаголом, и считаем, что это таргет предложения. Если вершина предложения не глагольная, то считаем, что в предложении таргета нет. Создаем таблицу sentences_and_targets, где есть столбцы с индексами предложений (sent_id), индексами текста (text_id), предложениями (text), токенизированными предложениями (tokenized_sentence), лемматизированными предложениями (lemmatixed_sentence), таргетами (target) и лемматизированными таргетами (lemma_target)

In [ ]:
def targets_to_dict(targets_path):
    df_targets = pd.read_csv(targets_path)
    targets_dict = {}
    targets_names = list(df_targets.target)
    targets_patterns = list(df_targets.ConstrPattern)
    verbs = []
    for name in targets_names:
        name = name.split(' ')
        verbs.append(name[0])
        
    for i in range(len(targets_names)):
        targets_dict[targets_names[i]] = [verbs[i], targets_patterns[i]]
    
    return targets_dict

In [ ]:
targets = targets_to_dict('targets.csv')

In [ ]:
targets['волноваться 1.1']

['волноваться', 'Snom V']

Функция, которая находит таргет в том случае, если его нет в списке таргетов

In [ ]:
def detect_target(text):
    match = re.findall(r'\tVERB\t_\t(.*?)\t0\troot\t_\t_\n', text)
    match = ' '.join(match)
    if (match != '') and ('\n' not in match):
        target = re.findall(r'\t(.*?)\t.*?\t0\troot\t_\t_\n', text)
        target = ' '.join(target)
        target_lemma = re.findall(target + '\t(.*?)\t', text)
        target_lemma = ' '.join(target_lemma)
        return target, target_lemma
    
    return '', ''

Функция, которая находит таргет предложения и его лемму.

In [ ]:
def preprocess_sentences(df, targets):
    texts_ud = list(df.text)
    sentences = []
    tokenized_sentences = []
    lemmatized_sentences = []
    sent_targets = []
    sent_lemm_targets = []
    for text in texts_ud:
        target = ''
        lemmas = []
        sent = re.findall(r'\n# text = (.*?)\n1\t', text)
        sent = ' '.join(sent)
        sentences.append(sent)
        sent = re.sub(r'[^\w^\s\']', '', sent)
        sent = sent.lower()
        tokenized_sent = word_tokenize(sent)
        tokenized_sentences.append(' '.join(tokenized_sent))
        for token in tokenized_sent:
            ana = morph.parse(token)
            ana = ana[0]
            lemma = ana.normal_form
            lemmas.append(lemma)
            for key, value in targets.items():
                if lemma == value[0]:
                    target = token
                    target_lemma = lemma
               
        if target == '':
            target, target_lemma = detect_target(text)
        sent_targets.append(target)
        sent_lemm_targets.append(target_lemma)    
        lemmatized_sentences.append(' '.join(lemmas))
    
    df['tokenized_sentence'] = tokenized_sentences
    df['lemmatixed_sentence'] = lemmatized_sentences
    df['target'] = sent_targets
    df['lemma_target'] = sent_lemm_targets
    return df

In [ ]:
sentences_and_targets = preprocess_sentences(df_framebank_ud, targets)
sentences_and_targets

,sent_id,text_id,text,tokenized_sentence,lemmatixed_sentence,target,lemma_target
0,0,4,# newdoc\n# newpar\n# sent_id = 1\n# text = Во...,вообще на процессе доминировали родственники у...,вообще на процесс доминировать родственник убитый,доминировали,доминировать
1,1,4,# sent_id = 2\n# text = Они называли Шахова зл...,они называли шахова злодеем и убийцей кровавым...,они называть шахов злодей и убийца кровавый чу...,называли,называть
2,2,4,# sent_id = 3\n# text = Ну а Пермяков с Пашути...,ну а пермяков с пашутиным ангелы небесные,ну а пермяк с пашутиный ангел небесный,NaN,NaN
3,3,4,# sent_id = 4\n# text = Никого они не могли на...,никого они не могли насиловать,никто они не мочь насиловать,могли,мочь
4,4,4,# sent_id = 5\n# text = Знай работали себе на ...,знай работали себе на местном предприятии из т...,знать работать себя на местный предприятие из ...,работали,работать
...,...,...,...,...,...,...,...
613809,613809,143528,# sent_id = 613810\n# text = Всё было когда-то...,всё было когдато и у меня,всё быть когдато и у я,было,быть
613810,613810,143528,# sent_id = 613811\n# text = Какие там контрол...,какие там контрольные,какой там контрольный,NaN,NaN
613811,613811,143528,"# sent_id = 613812\n# text = Он про отца-то, р...",он про отцато раскровенившего губу и думать забыл,он про отцато раскровенить губа и думать забыть,забыл,забыть
613812,613812,143528,# sent_id = 613813\n# text = Это было ясно.\n1...,это было ясно,это быть ясно,было,быть


# Идентификация участников

Для тех предложений, где были найдены таргеты, достаем те слова предложения, которые зависят от этого таргета. Достаем их синтаксические роли и морфологические признаки. Добавляем в таблицу sentences_and_targets столбцы с идентифицированными участниками (arguments), их синтаксическими ролями (synt_roles), частями речи (POS) и морфологическимми признаками (morph_features). Создаем новую таблицу arguments_syntax_morphology.

In [ ]:
def find_targets_id(df):
    df = df.dropna()
    list_targets_id = []
    list_all_arguments = []
    list_all_syntax_roles = []
    list_all_pos = []
    list_all_morph_features = []
    for index, row in df.iterrows():
        target = row['target']
        text = row['text']
        text_lower = text.lower()
        match = re.findall(r'\n(\d*?)\t' + target, text_lower)
        target_id = ' '.join(match)
        list_targets_id.append(target_id)
        list_arguments, list_syntax_roles, list_pos, list_morph_features = find_arguments(target_id, text)
        list_all_arguments.append(list_arguments)
        list_all_syntax_roles.append(list_syntax_roles)
        list_all_pos.append(list_pos)
        list_all_morph_features.append(list_morph_features)
    df['arguments'] = list_all_arguments
    df['synt_roles'] = list_all_syntax_roles
    df['POS'] = list_all_pos
    df['morph_features'] = list_all_morph_features
    return df

In [ ]:
def find_arguments(target_id, text):
    list_arguments = []
    list_syntax_roles = []
    list_pos = []
    list_morph_features = []
    match_word = re.findall(r'\n\d*?\t(.*?)\t.*?\t.*?\t_\t.*?\t' + target_id + '\t.*?\t.*?\t.*?', text)
    match_syntax = re.findall(r'\n\d*?\t.*?\t.*?\t.*?\t_\t.*?\t' + target_id + '\t(.*?)\t.*?\t.*?', text)
    match_pos = re.findall(r'\n\d*?\t.*?\t.*?\t(.*?)\t_\t.*?\t' + target_id + '\t.*?\t.*?\t.*?', text)
    match_morph = re.findall(r'\n\d*?\t.*?\t.*?\t.*?\t_\t(.*?)\t' + target_id + '\t.*?\t.*?\t.*?', text)
    for i in range(len(match_word)):
        if (match_syntax[i] != 'punct') and (match_syntax[i] != 'cc') and (match_syntax[i] != 'advmod'):
            list_arguments.append(match_word[i])
            list_syntax_roles.append(match_syntax[i])
            list_pos.append(match_pos[i])
            list_morph_features.append(match_morph[i])
    
    return list_arguments, list_syntax_roles, list_pos, list_morph_features

In [ ]:
arguments_syntax_morphology = find_targets_id(sentences_and_targets)
arguments_syntax_morphology

,sent_id,text_id,text,tokenized_sentence,lemmatixed_sentence,target,lemma_target,arguments,synt_roles,POS,morph_features
0,0,4,# newdoc\n# newpar\n# sent_id = 1\n# text = Во...,вообще на процессе доминировали родственники у...,вообще на процесс доминировать родственник убитый,доминировали,доминировать,"['процессе', 'родственники']","['obl', 'nsubj']","['NOUN', 'NOUN']",['Animacy=Inan|Case=Loc|Gender=Masc|Number=Sin...
1,1,4,# sent_id = 2\n# text = Они называли Шахова зл...,они называли шахова злодеем и убийцей кровавым...,они называть шахов злодей и убийца кровавый чу...,называли,называть,"['Они', 'Шахова', 'злодеем', 'чудовищем']","['nsubj', 'obj', 'obl', 'obl']","['PRON', 'PROPN', 'NOUN', 'NOUN']","['Case=Nom|Number=Plur|Person=3', 'Animacy=Ani..."
2,3,4,# sent_id = 4\n# text = Никого они не могли на...,никого они не могли насиловать,никто они не мочь насиловать,могли,мочь,"['они', 'насиловать']","['nsubj', 'xcomp']","['PRON', 'VERB']","['Case=Nom|Number=Plur|Person=3', 'Aspect=Imp|..."
3,4,4,# sent_id = 5\n# text = Знай работали себе на ...,знай работали себе на местном предприятии из т...,знать работать себя на местный предприятие из ...,работали,работать,"['себе', 'предприятии', 'тех']","['iobj', 'obl', 'det']","['PRON', 'NOUN', 'DET']","['Case=Dat', 'Animacy=Inan|Case=Loc|Gender=Neu..."
4,7,5,"# sent_id = 8\n# text = Пополз я, стало быть, ...",пополз я стало быть подальше а тут стонет ктото,поползти я стать быть далёкий а тут стонать ктоть,стонет,стонать,['кто-то'],['nsubj'],['PRON'],['Case=Nom']
...,...,...,...,...,...,...,...,...,...,...,...
510532,613807,143527,"# sent_id = 613808\n# text = Сказали, что в ме...",сказали что в мебельном возле коптевского рынк...,сказать что в мебельный возле коптевский рынок...,знает,знать,"['который', 'стол']","['nsubj', 'obl']","['PRON', 'NOUN']","['Case=Nom', 'Animacy=Inan|Case=Nom|Gender=Mas..."
510533,613808,143528,# sent_id = 613809\n# text = За калиткой их вс...,за калиткой их встречает собака на даче тепло ...,за калитка они встречать собака на дача тепло ...,горят,гореть,"['печке', 'дрова']","['obl', 'nsubj']","['NOUN', 'NOUN']",['Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing...
510534,613809,143528,# sent_id = 613810\n# text = Всё было когда-то...,всё было когдато и у меня,всё быть когдато и у я,было,быть,[],[],[],[]
510535,613811,143528,"# sent_id = 613812\n# text = Он про отца-то, р...",он про отцато раскровенившего губу и думать забыл,он про отцато раскровенить губа и думать забыть,забыл,забыть,['думать'],['xcomp'],['VERB'],['Aspect=Imp|VerbForm=Inf|Voice=Act']


Векторизуем таргеты, создаем новый датафрейм, где каждый участник предложения находится на отдельной строке, а синтаксические роли, части речи и морфологические признаки представлены в формате one-hot кодирования. Записываем получившийся датафрейм в таблицу data_srl. 

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
use_model = hub.load(module_url)
print("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder-multilingual/3 loaded


In [ ]:
def use_get_embedding(text, model):
    return model([text]).numpy()[0]

def use_records_to_embeds(list_text, model):
    embeddings = np.zeros((len(list_text), 512))
    for i in tq.tqdm(range(len(list_text))):
        embeddings[i] = use_get_embedding(list_text[i], model)
    return embeddings

In [ ]:
def make_df_with_embeddings(df): 
    arguments_per_sent = list(df.arguments)
    synt_roles_per_sent = list(df.synt_roles)
    pos_per_sent = list(df.POS)
    morph_per_sent = list(df.morph_features)
    targets_per_sent = list(df.lemma_target)
    sent_id_per_sent = list(df.sent_id)
    text_id_per_sent = list(df.text_id)
    arguments = []
    lemmas_targets = []
    sent_ids = []
    text_ids = []
    synt_roles = []
    pos_tags = []
    morph_tags = []
    counter = 0
    args_id = []
    for i in range(len(arguments_per_sent)):
        arguments_sent = ast.literal_eval(arguments_per_sent[i])
        synt_roles_sent = ast.literal_eval(synt_roles_per_sent[i])
        pos_sent = ast.literal_eval(pos_per_sent[i])
        targets_sent = targets_per_sent[i]
        sent_id_sent = sent_id_per_sent[i]
        text_id_sent = text_id_per_sent[i]
        morph_sent = ast.literal_eval(morph_per_sent[i])
        if (arguments_sent != []) and ('xcomp' not in synt_roles_sent)\
         and ('parataxis' not in synt_roles_sent)\
          and ('det' not in synt_roles_sent):
            for y in range(len(arguments_sent)):
                if (synt_roles_sent[y] != 'mark') and (pos_sent[y] != 'VERB')\
                 and (pos_sent[y] != 'AUX')\
                  and (synt_roles_sent[y] != 'advmod')\
                   and (morph_sent[y] != '_'):
                    arguments.append(arguments_sent[y])
                    lemmas_targets.append(targets_sent)
                    sent_ids.append(sent_id_sent)
                    text_ids.append(text_id_sent)
                    synt_roles.append(synt_roles_sent[y])
                    pos_tags.append(pos_sent[y])
                    morph = morph_sent[y]
                    tags = morph.split('|')
                    morph_tags.append(tags)
                    args_id.append(counter)
                    counter += 1

    use_embeddings_targets = use_records_to_embeds(lemmas_targets, use_model)
    new_df = pd.DataFrame() 
    new_df['argument_id'] = args_id
    new_df['argument'] = arguments
    new_df['text_id'] = text_ids
    new_df['sent_id'] = sent_ids

    list_use_embeddings_targets = []
    for embed in use_embeddings_targets:
        new_embed = embed.tolist()
        list_use_embeddings_targets.append(new_embed)
        
    new_df['target_lemma_embeddings'] = list_use_embeddings_targets
    new_df['synt_role'] = synt_roles
    new_df['pos'] = pos_tags
    
    dict_morph_features = {}
    unique_features = []
    for i in range(len(args_id)):
        dict_morph_features[args_id[i]] = []
        morph_features = feat.split('=')
        dict_morph_features[args_id[i]].append(morph_features[1])
        if morph_features[1] not in unique_features:
            unique_features.append(morph_features[1])
    
    dict_morphology = {}
    
    for feat in unique_features:
        dict_morphology[feat] = []
        for key, value in dict_morph_features.items():
            if feat in dict_morph_features[key]:
                dict_morphology[feat].append(1)
            else:
                dict_morphology[feat].append(0)
    
    for key, value in dict_morphology.items():
        new_df[key] = value
        
    columns = ['synt_role', 'pos']
    for column in columns:
        new_df = pd.get_dummies(new_df, columns=[column], prefix=column)
        
    columns_names2 = [i for i in range(512, 512 + len(list_use_embeddings_targets[0]))]
    col_1024_df = pd.DataFrame(list_use_embeddings_targets, columns=columns_names2)
    new_df = pd.concat([new_df, col_1024_df], axis=1)
    
    
    return new_df

In [ ]:
data_srl = make_df_with_embeddings(arguments_syntax_morphology)
data_srl

,argument_id,argument,text_id,sent_id,target_lemma_embeddings,Inan,Loc,Masc,Sing,Anim,Nom,Plur,3,Acc,Ins,Neut,1,Fem,Gen,Dat,Yes,Cmp,Pos,2,Imp,Ind,Pres,Fin,Act,Past,Short,Inf,Par,Sup,Conv,Voc,Cnd,Neg,synt_role_acl,synt_role_acl:relcl,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,процессе,4,0,"[0.12092537432909012, 0.019661663100123405, -0...",1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.015525,-0.034329,0.021721,0.042647,-0.040375,-0.056430,0.001357,0.043235,-0.052233,0.115407,0.042070,0.071383,-0.029100,-0.042942,0.037134,-0.104051,-0.013354,0.039081,0.033378,-0.011730,-0.034308,0.058822,0.010961,0.031681,-0.016150,-0.042789,-0.025703,-0.009317,-0.062688,-0.026092,-0.095414,0.018012,0.082838,0.009548,-0.013630,-0.039329,-0.036794,0.018246,-0.022193,0.056104
1,1,родственники,4,0,"[0.12092537432909012, 0.019661663100123405, -0...",0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.015525,-0.034329,0.021721,0.042647,-0.040375,-0.056430,0.001357,0.043235,-0.052233,0.115407,0.042070,0.071383,-0.029100,-0.042942,0.037134,-0.104051,-0.013354,0.039081,0.033378,-0.011730,-0.034308,0.058822,0.010961,0.031681,-0.016150,-0.042789,-0.025703,-0.009317,-0.062688,-0.026092,-0.095414,0.018012,0.082838,0.009548,-0.013630,-0.039329,-0.036794,0.018246,-0.022193,0.056104
2,2,Они,4,1,"[0.09806624054908752, 0.00872455071657896, 0.0...",0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.036977,0.055615,0.032634,0.050144,0.023175,-0.025106,0.034300,-0.004423,0.003330,0.089412,0.055677,-0.000704,-0.000181,0.018418,-0.022200,-0.067275,-0.044691,0.035548,0.017858,0.073380,0.042569,0.040674,-0.024042,-0.031585,-0.025270,-0.021858,-0.034752,0.062016,-0.072132,-0.012875,-0.074346,0.049717,0.057392,0.005131,0.009810,-0.033424,-0.000249,0.031347,0.034515,0.068631
3,3,Шахова,4,1,"[0.09806624054908752, 0.00872455071657896, 0.0...",0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.036977,0.055615,0.032634,0.050144,0.023175,-0.025106,0.034300,-0.004423,0.003330,0.089412,0.055677,-0.000704,-0.000181,0.018418,-0.022200,-0.067275,-0.044691,0.035548,0.017858,0.073380,0.042569,0.040674,-0.024042,-0.031585,-0.025270,-0.021858,-0.034752,0.062016,-0.072132,-0.012875,-0.074346,0.049717,0.057392,0.005131,0.009810,-0.033424,-0.000249,0.031347,0.034515,0.068631
4,4,злодеем,4,1,"[0.09806624054908752, 0.00872455071657896, 0.0...",1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.036977,0.055615,0.032634,0.050144,0.023175,-0.025106,0.034300,-0.004423,0.003330,0.089412,0.055677,-0.000704,-0.000181,0.018418,-0.022200,-0.067275,-0.044691,0.035548,0.017858,0.073380,0.042569,0.040674,-0.024042,-0.031585,-0.025270,-0.021858,-0.034752,0.062016,-0.072132,-0.012875,-0.074346,0.049717,0.057392,0.005131,0.009810,-0.033424,-0.000249,0.031347,0.034515,0.068631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637769,637769,Мариной,143527,613806,"[0.11125444620847702, -0.051691822707653046, 0...",1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.031952,-0.001695,-0.053384,0.042502,0.032877,0.000731,0.064203,0.010861,-0.021774,0.112390,0.051021,-0.016939,0.000700,-0.014022,-0.063526,-0.085583,-0.030890,-0.013960,0.007092,0.054002,0.002154,0.044394,-0.086923,0.013848,0.011350,-0.009266,0.044505,0.033535,-0.056034,-0.033534,-0.022630,-0.008438,0.056663,0.035566,0.016144,-0.000900,-0.002951,0.009078,-0.010650,0.058868
637770,637770,который,143527,613807,"[0.1266891062259674, -0.006970841437578201, 0....",0,0,0,0,0,1,0

Проходимся по каждому идентифицированному участнику предложения из таблицы data_srl и смотрим, есть ли этот участник в таблице "expl_framebank.csv". Если есть, то запоминаем его семантическую роль и записываем в новую таблицу data_srl_with_expl.

In [ ]:
def prepare_test_data(df_1, path):
    df_expl = pd.read_csv(path)
    examples_id = list(df_1.text_id)
    arguments = list(df_1.argument)
    arguments_from_fb = []
    expls_from_fb = []
    targets = []
    for example_id in examples_id:
        ex_id = int(example_id)
        arguments_list, expls = find_role_in_dataframe(ex_id, df_expl)
        arguments_from_fb.append(arguments_list)
        expls_from_fb.append(expls)
    
    for i in range(len(arguments)):
        arg = arguments[i]
        args_corr = arguments_from_fb[i]
        args_without_prep = []
        for arg_corr in args_corr:
            splitted_arg = (str(arg_corr)).split(' ')
            if len(splitted_arg) > 0:
                arg_corr = splitted_arg[-1]
            args_without_prep.append(str(arg_corr))
        if arg in args_without_prep:
            index_arg = args_without_prep.index(arg)
            targets.append(expls_from_fb[i][index_arg])
        else:
            targets.append(None)
    
    uniq_targets = []
    for target in tqdm(targets):
        if (target != None) and (target not in uniq_targets):
            uniq_targets.append(target)
    
    gold_expl = []
    for target in tqdm(targets):
        if target != None:
            gold_expl.append(uniq_targets.index(target))
        else:
            gold_expl.append(None)
            
    arguments_id = [i for i in range(len(targets))]
    dict_roles = {}
    for i in tqdm(range(len(arguments_id))):
        dict_roles[i] = gold_expl[i]
    
    gold_test = []
    gold_roles_test = []
    for index, row in tqdm(df_1.iterrows()):
        arg_id = row['argument_id']
        if arg_id in dict_roles.keys():
            role_id = dict_roles[arg_id]
            gold_test.append(role_id)
            gold_roles_test.append(targets[arg_id])
     
    
    df_1['Expl'] = gold_roles_test
    df_1['Expl_target'] = gold_test
    return df_1

In [ ]:
def find_role_in_dataframe(ex_id, df_expl):
    rslt_df = df_expl[df_expl['ExIndex'] == ex_id]
    arguments = list(rslt_df.WordDep)
    expls = list(rslt_df.Expl)
    return arguments, expls

In [ ]:
df_test_data = prepare_test_data(data_srl, 'expl_framebank.csv')
df_test_data

,argument_id,argument,text_id,sent_id,target_lemma_embeddings,Inan,Loc,Masc,Sing,Anim,Nom,Plur,3,Acc,Ins,Neut,1,Fem,Gen,Dat,Yes,Cmp,Pos,2,Imp,Ind,Pres,Fin,Act,Past,Short,Inf,Par,Sup,Conv,Voc,Cnd,Neg,synt_role_acl,synt_role_acl:relcl,...,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,Expl,Expl_target
0,0,процессе,4,0,"[0.12092537432909012, 0.019661663100123405, -0...",1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.021721,0.042647,-0.040375,-0.056430,0.001357,0.043235,-0.052233,0.115407,0.042070,0.071383,-0.029100,-0.042942,0.037134,-0.104051,-0.013354,0.039081,0.033378,-0.011730,-0.034308,0.058822,0.010961,0.031681,-0.016150,-0.042789,-0.025703,-0.009317,-0.062688,-0.026092,-0.095414,0.018012,0.082838,0.009548,-0.013630,-0.039329,-0.036794,0.018246,-0.022193,0.056104,NaN,NaN
1,1,родственники,4,0,"[0.12092537432909012, 0.019661663100123405, -0...",0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.021721,0.042647,-0.040375,-0.056430,0.001357,0.043235,-0.052233,0.115407,0.042070,0.071383,-0.029100,-0.042942,0.037134,-0.104051,-0.013354,0.039081,0.033378,-0.011730,-0.034308,0.058822,0.010961,0.031681,-0.016150,-0.042789,-0.025703,-0.009317,-0.062688,-0.026092,-0.095414,0.018012,0.082838,0.009548,-0.013630,-0.039329,-0.036794,0.018246,-0.022193,0.056104,NaN,NaN
2,2,Они,4,1,"[0.09806624054908752, 0.00872455071657896, 0.0...",0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.032634,0.050144,0.023175,-0.025106,0.034300,-0.004423,0.003330,0.089412,0.055677,-0.000704,-0.000181,0.018418,-0.022200,-0.067275,-0.044691,0.035548,0.017858,0.073380,0.042569,0.040674,-0.024042,-0.031585,-0.025270,-0.021858,-0.034752,0.062016,-0.072132,-0.012875,-0.074346,0.049717,0.057392,0.005131,0.009810,-0.033424,-0.000249,0.031347,0.034515,0.068631,NaN,NaN
3,3,Шахова,4,1,"[0.09806624054908752, 0.00872455071657896, 0.0...",0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.032634,0.050144,0.023175,-0.025106,0.034300,-0.004423,0.003330,0.089412,0.055677,-0.000704,-0.000181,0.018418,-0.022200,-0.067275,-0.044691,0.035548,0.017858,0.073380,0.042569,0.040674,-0.024042,-0.031585,-0.025270,-0.021858,-0.034752,0.062016,-0.072132,-0.012875,-0.074346,0.049717,0.057392,0.005131,0.009810,-0.033424,-0.000249,0.031347,0.034515,0.068631,NaN,NaN
4,4,злодеем,4,1,"[0.09806624054908752, 0.00872455071657896, 0.0...",1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.032634,0.050144,0.023175,-0.025106,0.034300,-0.004423,0.003330,0.089412,0.055677,-0.000704,-0.000181,0.018418,-0.022200,-0.067275,-0.044691,0.035548,0.017858,0.073380,0.042569,0.040674,-0.024042,-0.031585,-0.025270,-0.021858,-0.034752,0.062016,-0.072132,-0.012875,-0.074346,0.049717,0.057392,0.005131,0.009810,-0.033424,-0.000249,0.031347,0.034515,0.068631,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637769,637769,Мариной,143527,613806,"[0.11125444620847702, -0.051691822707653046, 0...",1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.053384,0.042502,0.032877,0.000731,0.064203,0.010861,-0.021774,0.112390,0.051021,-0.016939,0.000700,-0.014022,-0.063526,-0.085583,-0.030890,-0.013960,0.007092,0.054002,0.002154,0.044394,-0.086923,0.013848,0.011350,-0.009266,0.044505,0.033535,-0.056034,-0.033534,-0.022630,-0.008438,0.056663,0.035566,0.016144,-0.000900,-0.002951,0.009078,-0.010650,0.058868,NaN,NaN
637770,637770,который,143527,613807,"[0.1266891062259674, -0.006970841437578201, 0....",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...

Записываем правильно определенных участников в таблицу data_srl_with_expl_without_none.

In [ ]:
def correct_identified(df_test_data):  
    data_srl_corr = df_test_data.dropna()
    list_expl = list(data_srl_corr.Expl)
    data_srl_corr = data_srl_corr[data_srl_corr['Expl'] != '-']
    data_srl_corr = data_srl_corr[data_srl_corr['Expl'] != '?']
    return data_srl_corr

In [ ]:
data_srl_with_expl_without_none = correct_identified(df_test_data)
data_srl_with_expl_without_none

,argument_id,argument,text_id,sent_id,target_lemma_embeddings,Inan,Loc,Masc,Sing,Anim,Nom,Plur,3,Acc,Ins,Neut,1,Fem,Gen,Dat,Yes,Cmp,Pos,2,Imp,Ind,Pres,Fin,Act,Past,Short,Inf,Par,Sup,Conv,Voc,Cnd,Neg,synt_role_acl,synt_role_acl:relcl,...,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,Expl,Expl_target
0,7,я,5,9,"[0.11169246584177017, 0.0034639909863471985, -...",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.009054,0.036962,-0.027880,0.009755,0.044076,0.050473,-0.010219,0.109224,0.072542,0.010778,-0.023289,-0.029202,0.031587,-0.095663,-0.030265,0.014959,-0.005330,0.033569,0.017686,0.043723,-0.023228,0.014803,0.003874,-0.008340,-0.050731,-0.043590,-0.068581,-0.048679,-0.087321,0.015323,0.066811,0.019672,-0.031053,0.022285,-0.030707,0.046705,0.010773,0.087052,агенс,0.0
1,12,я,7,17,"[0.1266891062259674, -0.006970841437578201, 0....",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.052134,0.022388,0.023189,0.042598,0.058372,0.031449,0.084169,0.112273,0.061660,0.016715,-0.006281,0.028238,-0.013885,-0.101738,-0.008863,0.024120,0.034087,0.016196,0.002407,0.065306,0.006054,0.006917,-0.007877,-0.016010,0.059870,0.053123,0.062172,-0.011848,-0.096993,0.023920,0.042821,-0.054780,-0.002103,-0.010365,-0.063280,0.011680,0.032388,0.057121,субъект ментального состояния,1.0
2,43,массы,1004,45,"[0.09921302646398544, 0.009834939613938332, -0...",1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000064,-0.029069,-0.030072,-0.069320,0.013085,-0.083599,0.027731,0.095086,0.044861,-0.021444,-0.033710,0.018005,0.007025,-0.089738,-0.041497,-0.033367,0.025126,0.006846,-0.023778,-0.002014,-0.067092,-0.016213,-0.067004,-0.013016,-0.063353,-0.030804,-0.072955,-0.020225,-0.006737,0.019626,0.071582,0.031376,0.026577,-0.070713,0.043433,0.041253,0.065653,0.037860,субъект поведения,2.0
3,53,я,1007,59,"[0.11409293115139008, -0.014948656782507896, -...",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.021242,0.051875,0.003824,-0.024389,0.013670,0.047655,-0.003194,0.110252,0.053915,0.065645,-0.010941,0.051255,-0.002205,-0.093023,0.016876,0.030711,0.058615,0.028691,0.073298,-0.004446,0.045160,-0.016358,-0.072087,-0.051022,0.002213,0.033495,-0.041125,0.042027,-0.065245,0.021334,0.051135,-0.025646,-0.033370,0.001352,0.015450,0.091039,0.047637,0.091847,субъект психологического состояния,3.0
4,61,некому,1010,69,"[0.09921302646398544, 0.009834939613938332, -0...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000064,-0.029069,-0.030072,-0.069320,0.013085,-0.083599,0.027731,0.095086,0.044861,-0.021444,-0.033710,0.018005,0.007025,-0.089738,-0.041497,-0.033367,0.025126,0.006846,-0.023778,-0.002014,-0.067092,-0.016213,-0.067004,-0.013016,-0.063353,-0.030804,-0.072955,-0.020225,-0.006737,0.019626,0.071582,0.031376,0.026577,-0.070713,0.043433,0.041253,0.065653,0.037860,субъект психологического состояния,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18986,637674,все,143512,613713,"[0.10206297785043716, -0.05749831721186638, -0...",0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.010677,-0.003299,0.031774,-0.024021,0.003462,0.029202,0.050685,0.102922,0.039601,0.026846,-0.008470,0.052986,0.015462,-0.093602,-0.052782,0.032749,0.068341,0.061128,-0.028418,0.009667,-0.066700,0.048813,-0.030430,-0.005111,-0.063871,0.053783,-0.080793,-0.079926,-0.048520,-0.004638,0.049407,-0.012345,0.037936,0.050667,0.001860,0.060721,0.072273,0.071314,субъект ментального состояния,1.0
18987,637685,вы,143514,613727,"[0.10206297785043

Считаем самые частотные роли.

In [ ]:
def count_freq(data):
    list_expl = list(data.Expl)
    counter = Counter(list_expl)
    counter_2 = counter.most_common()
    keys = []
    values = []
    for pair in counter_2:
        keys.append(pair[0])
        values.append(pair[1])
    df_counter = pd.DataFrame()
    df_counter['expl'] = keys
    df_counter['frequency'] = values
    return df_counter

In [ ]:
freq = count_freq(data_srl_with_expl_without_none)
freq

,expl,frequency
0,пациенс,2106
1,агенс,1715
2,тема,1334
3,субъект перемещения,1056
4,субъект психологического состояния,997
...,...,...
209,конечная точка - статус,1
210,ситуация в фокусе - стимул,1
211,субъект восприятия\r\n,1
212,источник звука - часть говорящего,1


Упрощаем список семантических ролей фреймбанка, объединяя разные подвиды семантических ролей в один вид. Записываем новую таблицу data_srl_with_changed_expl.

In [ ]:
def change_expl(df):
    old_expl = list(df.Expl)
    new_expl = []
    expls_to_change = ['агенс', 'адресат', 'говорящий', 'запах', 'значение параметра', 'инструмент', 'источник', 'качество',
                     'конечная точка', 'контрагент', 'место', 'момент', 'направление', 'начальная точка', 'опора', 
                     'пациенс', 'потенциальная угроза', 'предмет высказывания', 'предмет мысли', 'признак', 'расстояние',
                     'результат', 'ресурс', 'свойство причины', 'ситуация', 'содержание', 'средство', 'статус', 'стимул', 
                     'субъект', 'цель', 'часть субъект', 'эталон']

    for expl_df in old_expl:
        ex = expl_df
        for expl_new in expls_to_change:
            if expl_new in expl_df:
                ex = expl_new
        new_expl.append(ex)   

    df['Expl_changed'] = new_expl
    unique_expl = []
    for expl in new_expl:
        if expl not in unique_expl:
            unique_expl.append(expl)

    index_expl = []
    for expl in new_expl:
        index_expl.append(unique_expl.index(expl))

    df['Expl_target_changed'] = index_expl
    df = df[df['Expl_changed'] != '-']
    df = df[df['Expl_changed'] != '?']
    return df

In [ ]:
data_srl_with_changed_expl = change_expl(data_srl_with_expl_without_none)
data_srl_with_changed_expl

,argument_id,argument,text_id,sent_id,target_lemma_embeddings,Inan,Loc,Masc,Sing,Anim,Nom,Plur,3,Acc,Ins,Neut,1,Fem,Gen,Dat,Yes,Cmp,Pos,2,Imp,Ind,Pres,Fin,Act,Past,Short,Inf,Par,Sup,Conv,Voc,Cnd,Neg,synt_role_acl,synt_role_acl:relcl,...,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,Expl,Expl_target,Expl_changed,Expl_target_changed
0,7,я,5,9,"[0.11169246584177017, 0.0034639909863471985, -...",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.027880,0.009755,0.044076,0.050473,-0.010219,0.109224,0.072542,0.010778,-0.023289,-0.029202,0.031587,-0.095663,-0.030265,0.014959,-0.005330,0.033569,0.017686,0.043723,-0.023228,0.014803,0.003874,-0.008340,-0.050731,-0.043590,-0.068581,-0.048679,-0.087321,0.015323,0.066811,0.019672,-0.031053,0.022285,-0.030707,0.046705,0.010773,0.087052,агенс,0.0,агенс,0
1,12,я,7,17,"[0.1266891062259674, -0.006970841437578201, 0....",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.023189,0.042598,0.058372,0.031449,0.084169,0.112273,0.061660,0.016715,-0.006281,0.028238,-0.013885,-0.101738,-0.008863,0.024120,0.034087,0.016196,0.002407,0.065306,0.006054,0.006917,-0.007877,-0.016010,0.059870,0.053123,0.062172,-0.011848,-0.096993,0.023920,0.042821,-0.054780,-0.002103,-0.010365,-0.063280,0.011680,0.032388,0.057121,субъект ментального состояния,1.0,субъект,1
2,43,массы,1004,45,"[0.09921302646398544, 0.009834939613938332, -0...",1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.030072,-0.069320,0.013085,-0.083599,0.027731,0.095086,0.044861,-0.021444,-0.033710,0.018005,0.007025,-0.089738,-0.041497,-0.033367,0.025126,0.006846,-0.023778,-0.002014,-0.067092,-0.016213,-0.067004,-0.013016,-0.063353,-0.030804,-0.072955,-0.020225,-0.006737,0.019626,0.071582,0.031376,0.026577,-0.070713,0.043433,0.041253,0.065653,0.037860,субъект поведения,2.0,субъект,1
3,53,я,1007,59,"[0.11409293115139008, -0.014948656782507896, -...",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.003824,-0.024389,0.013670,0.047655,-0.003194,0.110252,0.053915,0.065645,-0.010941,0.051255,-0.002205,-0.093023,0.016876,0.030711,0.058615,0.028691,0.073298,-0.004446,0.045160,-0.016358,-0.072087,-0.051022,0.002213,0.033495,-0.041125,0.042027,-0.065245,0.021334,0.051135,-0.025646,-0.033370,0.001352,0.015450,0.091039,0.047637,0.091847,субъект психологического состояния,3.0,субъект,1
4,61,некому,1010,69,"[0.09921302646398544, 0.009834939613938332, -0...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.030072,-0.069320,0.013085,-0.083599,0.027731,0.095086,0.044861,-0.021444,-0.033710,0.018005,0.007025,-0.089738,-0.041497,-0.033367,0.025126,0.006846,-0.023778,-0.002014,-0.067092,-0.016213,-0.067004,-0.013016,-0.063353,-0.030804,-0.072955,-0.020225,-0.006737,0.019626,0.071582,0.031376,0.026577,-0.070713,0.043433,0.041253,0.065653,0.037860,субъект психологического состояния,4.0,субъект,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18986,637674,все,143512,613713,"[0.10206297785043716, -0.05749831721186638, -0...",0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.031774,-0.024021,0.003462,0.029202,0.050685,0.102922,0.039601,0.026846,-0.008470,0.052986,0.015462,-0.093602,-0.052782,0.032749,0.068341,0.061128,-0.028418,0.009667,-0.066700,0.048813,-0.030430,-0.005111,-0.063871,0.053783,-0.080793,-0.079926,-0.048520,-0.004638,0.049407,-0.012345,0.037936,0.050667,0.001860,0.060721,0.072273,0.071314,субъект ментального состояния,1.0,субъект,1
18987,637685,вы,143514,613727,"[0.10206297785043716, -0.05749831721186638, -0..

Считаем частотность измененных меток.

In [ ]:
def count_freq_changed_expl(df):
    new_expl = df.Expl_changed
    new_expl_freq = Counter(new_expl)
    new_expl_df = pd.DataFrame()
    most_common_counter = new_expl_freq.most_common()
    keys = []
    values = []
    for expl in most_common_counter:
        keys.append(expl[0])
        values.append(expl[1])

    new_expl_df['expl'] = keys
    new_expl_df['frequency'] = values
    return new_expl_df

In [ ]:
expl_frequency_changed = count_freq_changed_expl(data_srl_with_changed_expl)
expl_frequency_changed

,expl,frequency
0,субъект,3327
1,пациенс,3055
2,агенс,1778
3,тема,1334
4,содержание,1234
...,...,...
71,имя существительное,1
72,начальный посессор,1
73,мера,1
74,малефициант,1


Создадим таблицу, где будут только те участники, которые попали в топ-9 самых частотных семантических ролей.

In [ ]:
def top_9_expl(df):
    list_top = ['субъект', 'пациенс', 'агенс', 'тема',
                'содержание', 'причина', 'место', 'говорящий',
                'конечная точка']
    for index, row in tqdm(df.iterrows()):
        if row['Expl_changed'] not in list_top:
            df.drop(index, inplace=True)

    return df

In [ ]:
top_9 = top_9_expl(data_srl_with_changed_expl)
top_9

,argument_id,argument,text_id,sent_id,target_lemma_embeddings,Inan,Loc,Masc,Sing,Anim,Nom,Plur,3,Acc,Ins,Neut,1,Fem,Gen,Dat,Yes,Cmp,Pos,2,Imp,Ind,Pres,Fin,Act,Past,Short,Inf,Par,Sup,Conv,Voc,Cnd,Neg,synt_role_acl,synt_role_acl:relcl,...,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,Expl,Expl_target,Expl_changed,Expl_target_changed
0,7,я,5,9,"[0.11169246584177017, 0.0034639909863471985, -...",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.027880,0.009755,0.044076,0.050473,-0.010219,0.109224,0.072542,0.010778,-0.023289,-0.029202,0.031587,-0.095663,-0.030265,0.014959,-0.005330,0.033569,0.017686,0.043723,-0.023228,0.014803,0.003874,-0.008340,-0.050731,-0.043590,-0.068581,-0.048679,-0.087321,0.015323,0.066811,0.019672,-0.031053,0.022285,-0.030707,0.046705,0.010773,0.087052,агенс,0.0,агенс,0
1,12,я,7,17,"[0.1266891062259674, -0.006970841437578201, 0....",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.023189,0.042598,0.058372,0.031449,0.084169,0.112273,0.061660,0.016715,-0.006281,0.028238,-0.013885,-0.101738,-0.008863,0.024120,0.034087,0.016196,0.002407,0.065306,0.006054,0.006917,-0.007877,-0.016010,0.059870,0.053123,0.062172,-0.011848,-0.096993,0.023920,0.042821,-0.054780,-0.002103,-0.010365,-0.063280,0.011680,0.032388,0.057121,субъект ментального состояния,1.0,субъект,1
2,43,массы,1004,45,"[0.09921302646398544, 0.009834939613938332, -0...",1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.030072,-0.069320,0.013085,-0.083599,0.027731,0.095086,0.044861,-0.021444,-0.033710,0.018005,0.007025,-0.089738,-0.041497,-0.033367,0.025126,0.006846,-0.023778,-0.002014,-0.067092,-0.016213,-0.067004,-0.013016,-0.063353,-0.030804,-0.072955,-0.020225,-0.006737,0.019626,0.071582,0.031376,0.026577,-0.070713,0.043433,0.041253,0.065653,0.037860,субъект поведения,2.0,субъект,1
3,53,я,1007,59,"[0.11409293115139008, -0.014948656782507896, -...",0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.003824,-0.024389,0.013670,0.047655,-0.003194,0.110252,0.053915,0.065645,-0.010941,0.051255,-0.002205,-0.093023,0.016876,0.030711,0.058615,0.028691,0.073298,-0.004446,0.045160,-0.016358,-0.072087,-0.051022,0.002213,0.033495,-0.041125,0.042027,-0.065245,0.021334,0.051135,-0.025646,-0.033370,0.001352,0.015450,0.091039,0.047637,0.091847,субъект психологического состояния,3.0,субъект,1
4,61,некому,1010,69,"[0.09921302646398544, 0.009834939613938332, -0...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.030072,-0.069320,0.013085,-0.083599,0.027731,0.095086,0.044861,-0.021444,-0.033710,0.018005,0.007025,-0.089738,-0.041497,-0.033367,0.025126,0.006846,-0.023778,-0.002014,-0.067092,-0.016213,-0.067004,-0.013016,-0.063353,-0.030804,-0.072955,-0.020225,-0.006737,0.019626,0.071582,0.031376,0.026577,-0.070713,0.043433,0.041253,0.065653,0.037860,субъект психологического состояния,4.0,субъект,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13833,637673,ней,143512,613713,"[0.10206297785043716, -0.05749831721186638, -0...",0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.031774,-0.024021,0.003462,0.029202,0.050685,0.102922,0.039601,0.026846,-0.008470,0.052986,0.015462,-0.093602,-0.052782,0.032749,0.068341,0.061128,-0.028418,0.009667,-0.066700,0.048813,-0.030430,-0.005111,-0.063871,0.053783,-0.080793,-0.079926,-0.048520,-0.004638,0.049407,-0.012345,0.037936,0.050667,0.001860,0.060721,0.072273,0.071314,содержание мысли,14.0,содержание,8
13834,637674,все,143512,613713,"[0.10206297785043716, -0.05749831721186638, -0...",0,0,0

# Кластеризация

### AgglomerativeClustering

#### 1) Бейзлайн. 
Для всех участников, у которых есть семантическая разметка, делаем кластеризацию. В качестве признаков - морфосинтаксические признаки и индекс позиции в стандартной реализации. FMI = TP / sqrt((TP + FP) * (TP + FN)) = 0.17

In [ ]:
def prep_baseline_data():
    df_baseline = pd.read_csv('baseline_df_4.csv')
    df_baseline.dropna(inplace=True)
    del df_baseline['Unnamed: 0']
    columns = ['ItemDep', 'Place']
    for column in columns:
        df_baseline = pd.get_dummies(df_baseline, columns=[column], prefix=column)

    expl_base = list(df_baseline.Expl)
    unique_b = []
    for expl in expl_base:
        if expl not in unique_b:
            unique_b.append(expl)

    id_base = []
    for expl in expl_base:
        id_base.append(unique_b.index(expl))

    df_baseline['Expl_target'] = id_base
    return df_baseline

In [ ]:
baseline_feat = prep_baseline_data()
baseline_feat

,ItemIndex,WordDep,Expl,ItemDep_ Aacc,ItemDep_ SPROdat,"ItemDep_""CL""",ItemDep_-,ItemDep_-> Sgen,ItemDep_A,ItemDep_ADV,ItemDep_ADV + Sgen,ItemDep_ADV / в + Sacc,ItemDep_ADVP,ItemDep_ADVPRO,ItemDep_ADVcomp,ItemDep_ANUM,ItemDep_ANUMacc,ItemDep_ANUMdat,ItemDep_ANUMgen,ItemDep_ANUMins,ItemDep_ANUMnom,ItemDep_APRO,ItemDep_APROacc,ItemDep_APROdat,ItemDep_APROgen,ItemDep_APROins,ItemDep_APROloc,ItemDep_APROnom,ItemDep_Aacc,ItemDep_Abrev,ItemDep_Acomp,ItemDep_Adat,ItemDep_Agen,ItemDep_Ains,ItemDep_Ains + Sins,ItemDep_Ains + слезами,ItemDep_Aloc,ItemDep_Anom,ItemDep_CL,ItemDep_CLi,...,ItemDep_сбоку от +Sgen,ItemDep_сверх + SPROgen,ItemDep_своим + Sdat,ItemDep_свыше + NUMgen,ItemDep_себя,ItemDep_сквозь + APROacc,ItemDep_сквозь + Sacc,ItemDep_сколько + CL,ItemDep_согласно + Sdat,ItemDep_среди + APROgen,ItemDep_среди + SPROgen,ItemDep_среди + Sgen,ItemDep_тем + что + CL,"ItemDep_тем, что + Cl",ItemDep_то + что + CL,"ItemDep_тому, что + CL",ItemDep_тревогу,ItemDep_у + APROgen,ItemDep_у + NUMgen,ItemDep_у + SPROgen,ItemDep_у + Sgen,ItemDep_у +APROgen,ItemDep_у +SPROgen,ItemDep_час,ItemDep_через + NUMacc,ItemDep_через + Sacc,ItemDep_что,ItemDep_что + CL,ItemDep_чтоб + CL,ItemDep_чтобы + CL,ItemDep_чтобы + Vinf,ItemDep_язык,Place_0,Place_1,Place_2,Place_3,Place_4,Place_5,Place_6,Expl_target
0,2,конвой,агенс,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,5,его,пациенс перемещения,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,72,с балкона,начальная точка,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2
3,2,родителей,субъект социального отношения,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3
4,5,чадо,пациенс социального отношения,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53069,5,Олечку,стимул,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,20
53070,2,Он,субъект восприятия,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,45
53071,6,прямо,способ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,25
53072,2,Мама,субъект восприятия,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,45


In [ ]:
len(Counter(list(baseline_feat.Expl))) #количество кластеров

296

In [ ]:
def clustering_AgglomerativeClustering(df):
    targets_int = df['Expl_target'].to_numpy()
    all_columns = list(df.columns)
    target = 'Expl_target'
    none_columns = ['ItemIndex', 'WordDep', 'Expl_target', 'Expl']

    feature_columns = []
    for col in all_columns:
        if col not in none_columns:
            feature_columns.append(col)

    df_feat = df[feature_columns]
    clustering_model = AgglomerativeClustering(
          n_clusters=296,
          distance_threshold=None,
          linkage="average",
          affinity="cosine"
      )
  
    start_time = datetime.now()
    clustering_model.fit(df_feat)
    print('Затраченное время: ', datetime.now() - start_time)
    predictions = clustering_model.labels_   
    metric2 = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('fowlkes_mallows_score: ', metric2)
    return predictions, targets_int

In [ ]:
predictions_b, targets_int_b = clustering_AgglomerativeClustering(baseline_feat)

0:09:35.274877
fowlkes_mallows_score:  0.17342187290627015


#### 2) Кластеризация на правильно идентифицировнных данных.

На основном наборе данных с эмбеддингами таргетов.

FMI = TP / sqrt((TP + FP) * (TP + FN)) = 0.1

In [ ]:
print(len(Counter(list(data_srl_with_changed_expl.Expl)))) #количество кластеров

214


In [ ]:
def clustering_AgglomerativeClustering_1(df):
    targets_int = df['Expl_target'].to_numpy()
    all_columns = list(df.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id', 
                     'word_embeddings', 'target_lemma_embeddings', 'Expl', 
                     'Expl_target', 'Expl_changed', 'Expl_target_changed'] 

    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df[feature_columns]
    start_time = datetime.now()
    clustering_model = AgglomerativeClustering(
            n_clusters=214,
            distance_threshold=None,
            linkage="average",
            affinity="cosine"
        )
  
    clustering_model.fit(df_feat)
    print('Затраченное время: ', datetime.now() - start_time)
    predictions = clustering_model.labels_ 
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int1, predictions1 = clustering_AgglomerativeClustering_1(data_srl_with_changed_expl)

Затраченное время:  0:01:52.446951
FMI:  0.09714499799432211


На основном наборе данных без эмбеддингов таргетов. FMI = TP / sqrt((TP + FP) * (TP + FN)) = 0.08

In [ ]:
def clustering_AgglomerativeClustering_2(df):
    targets_int = df['Expl_target'].to_numpy()
    all_columns = list(df.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id',
                     'word_embeddings', 'target_lemma_embeddings', 'Expl', 
                     'Expl_target', 'Expl_changed', 'Expl_target_changed'] 
    for i in range(512, 1024):
        none_features.append(str(i))
    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df[feature_columns]
    start_time = datetime.now()
    clustering_model = AgglomerativeClustering(
            n_clusters=214,
            distance_threshold=None,
            linkage="average",
            affinity="cosine"
        )
  
    clustering_model.fit(df_feat)
    print('Затраченное время: ', datetime.now() - start_time)
    predictions = clustering_model.labels_ 
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int_2, predictions_2

In [ ]:
targets_int2, predictions2 = clustering_AgglomerativeClustering_2(data_srl_with_changed_expl)

Затраченное время:  0:00:17.039267
FMI:  0.08380865541995797


На основном наборе данных с измененными метками с эмбеддингами FMI = TP / sqrt((TP + FP) * (TP + FN)) = 0.2

In [ ]:
def clustering_AgglomerativeClustering_3(df):
    targets_int = df['Expl_target_changed'].to_numpy()
    all_columns = list(df.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id', 'word_embeddings', 'target_lemma_embeddings',
                     'Expl', 'Expl_target', 'Expl_changed', 'Expl_target_changed'] 

    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df[feature_columns]
    start_time = datetime.now()
    clustering_model = AgglomerativeClustering(
            n_clusters=75,
            distance_threshold=None,
            linkage="average",
            affinity="cosine"
        )
  
    clustering_model.fit(df_feat)
    print('Затраченное время: ', datetime.now() - start_time)
    predictions = clustering_model.labels_ 
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int3, predictions3 = clustering_AgglomerativeClustering_3(data_srl_with_changed_expl)

Затраченное время:  0:01:50.238025
FMI:  0.19697166082856427


На основном наборе данных с измененными метками без эмбеддингов. FMI = TP / sqrt((TP + FP) * (TP + FN)) = 0.17

In [ ]:
def clustering_AgglomerativeClustering_4(df):
    targets_int = df['Expl_target_changed'].to_numpy()
    all_columns = list(df.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id',
                     'word_embeddings', 'target_lemma_embeddings', 'Expl',
                     'Expl_target', 'Expl_changed', 'Expl_target_changed'] 
    for i in range(512, 1024):
        none_features.append(str(i))
    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df[feature_columns]
    start_time = datetime.now()
    clustering_model = AgglomerativeClustering(
            n_clusters=75,
            distance_threshold=None,
            linkage="average",
            affinity="cosine"
        )
  
    clustering_model.fit(df_feat)
    print('Затраченное время: ', datetime.now() - start_time)
    predictions = clustering_model.labels_   
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int4, predictions4 = clustering_AgglomerativeClustering_4(data_srl_with_changed_expl)

Затраченное время:  0:00:16.616517
FMI:  0.17141696483771798


На основных данных с метаролями с эмбеддингами. FMI = 0.37

In [ ]:
def clustering_AgglomerativeClustering_5(df):
    targets_int = df['Expl_target_changed'].to_numpy()
    all_columns = list(df.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id',
                     'word_embeddings', 'target_lemma_embeddings',
                     'Expl', 'Expl_target', 'Expl_changed',
                     'Expl_target_changed'] 

    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df[feature_columns]
    start_time = datetime.now()
    clustering_model = AgglomerativeClustering(
            n_clusters=9,
            distance_threshold=None,
            linkage="average",
            affinity="cosine"
        )
  
    clustering_model.fit(df_feat)
    print('Затраченное время: ', datetime.now() - start_time)
    predictions = clustering_model.labels_
    print('Кол-во кластеров: ', clustering_model.n_clusters_)   
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int5, predictions5 = clustering_AgglomerativeClustering_5(top_9)

Затраченное время:  0:00:48.075217
FMI:  0.3735492239776036


На досновном наборе данных с метаролями без эмбеддингов. FMI = 0.35

In [ ]:
def clustering_AgglomerativeClustering_6(df):
    targets_int = df['Expl_target_changed'].to_numpy()
    all_columns = list(df.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id',
                     'word_embeddings', 'target_lemma_embeddings',
                     'Expl', 'Expl_target', 'Expl_changed',
                     'Expl_target_changed'] 

    for i in range(512, 1024):
        none_features.append(str(i))
    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df[feature_columns]
    start_time = datetime.now()
    clustering_model = AgglomerativeClustering(
            n_clusters=9,
            distance_threshold=None,
            linkage="average",
            affinity="cosine"
        )
  
    clustering_model.fit(df_feat)
    print('Затраченное время: ', datetime.now() - start_time)
    predictions = clustering_model.labels_  
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int_6, predictions_6 = clustering_AgglomerativeClustering_6(top_9)

Затраченное время:  0:00:07.176103
FMI:  0.3531280205197866


# K-means

Здесь кластеризатор обучается на обучающем наборе данных. В признаки для обучения входят эмбеддинги таргетов.

Количество кластеров = 214. Тест на основном наборе данных. FMI = TP / sqrt((TP + FP) * (TP + FN)) = 0.07

In [ ]:
def make_train_df():
    arguments_ids = data_srl_with_changed_expl.argument_id
    for arg_id in arguments_ids:
        df_to_cluster.drop(df_to_cluster[df_to_cluster['argument_id'] == arg_id].index, inplace = True)
    return df_to_cluster

In [ ]:
df_train = make_train_df()

In [ ]:
def clustering_kmeans_1(df_train, df_test):
    targets = list(df_test.Expl_target)
    targets_int = [int(i) for i in targets]
    all_columns = list(df_train.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id',
                     'word_embeddings', 'target_lemma_embeddings', 'Expl',
                     'Expl_target', 'Expl_changed', 'Expl_target_changed'] 
  
    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df_train[feature_columns]
    df_test = df_test[feature_columns]
    start_time = datetime.now()
    kmeans = KMeans(n_clusters=214).fit(df_feat)
    predictions = kmeans.predict(df_test)
    print('Затраченное время: ', datetime.now() - start_time)   
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int_k1, predictions_k1 = clustering_kmeans_1(df_train, data_srl_with_changed_expl)

Затраченное время:  0:44:55.683782
FMI:  0.07192557392856652


Количество кластеров = 75. Тест на основном наборе данных с измененными метками. FMI = 0.12

In [ ]:
def clustering_kmeans_2(df_train, df_test):
    targets = list(df_test.Expl_target_changed)
    targets_int = [int(i) for i in targets]
    all_columns = list(df_train.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id',
                     'word_embeddings', 'target_lemma_embeddings', 'Expl',
                     'Expl_target', 'Expl_changed', 'Expl_target_changed'] 
  
    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df_train[feature_columns]
    df_test = df_test[feature_columns]
    start_time = datetime.now()
    kmeans = KMeans(n_clusters=75).fit(df_feat)
    predictions = kmeans.predict(df_test)
    print('Затраченное время: ', datetime.now() - start_time)   
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int_k2, predictions_k2 = clustering_kmeans_2(df_train, data_srl_with_changed_expl)

Затраченное время:  0:15:00.796682
FMI:  0.11588909663148242


Количество кластеров = 9. Тест на основном наборе данных с метаролями. 

In [ ]:
def clustering_kmeans_3(df_train, df_test):
    targets = list(df_test.Expl_target_changed)
    targets_int = [int(i) for i in targets]
    all_columns = list(df_train.columns)
    none_features = ['argument_id', 'argument', 'text_id', 'sent_id',
                     'word_embeddings', 'target_lemma_embeddings', 'Expl',
                     'Expl_target', 'Expl_changed', 'Expl_target_changed']
    
    feature_columns = []
    for col in all_columns:
        if col not in none_features:
            feature_columns.append(col)  

    df_feat = df_train[feature_columns]
    df_test = df_test[feature_columns]
    start_time = datetime.now()
    kmeans = KMeans(n_clusters=9).fit(df_feat)
    predictions = kmeans.predict(df_test)
    print('Затраченное время: ', datetime.now() - start_time)   
    metric = metrics.fowlkes_mallows_score(targets_int, predictions) 
    print('FMI: ', metric)
    return targets_int, predictions

In [ ]:
targets_int_k3, predictions_k3 = clustering_kmeans_3(df_train, top_9)

Затраченное время:  0:05:07.572428
FMI:  0.2654887024128639
